In [1]:
import yaml
from pathlib import Path
import cv2
from matplotlib import pyplot as plt
import json
import math
import sys


In [14]:
conf = yaml.safe_load(Path('./train.yaml').read_text())
threhold = 20  #drop small images
img_id = 0
for i in conf:
    if i['boxes'] != []:
        img = cv2.imread(i['path'],1)
        for data in i['boxes']:
            data['x_max']=min(data['x_max'],1279)
            data['x_min']=max(data['x_min'],0)
            data['y_max']=min(data['y_max'],719)
            data['y_min']=max(data['y_min'],0)
            if data['x_max']-data['x_min'] >= threhold or data['y_max']-data['y_min'] >= threhold:
                try:
                    img_cut = cv2.resize(img[int(data['y_min']):math.ceil(data['y_max']),int(data['x_min']):math.ceil(data['x_max']),:],(64,64),interpolation = cv2.INTER_LINEAR)
                except:
                    print(int(data['y_min']),math.ceil(data['y_max']),int(data['x_min']),math.ceil(data['x_max']))
                    sys.exit()
                filename = f'./train_data/{img_id}.jpg'
                cv2.imwrite(filename, img_cut) 
                dic = {
                    img_id:data['label']
                }
                try:
                    with open("train_label.json", "r") as old_file:
                        content = json.load(old_file)
                        content.update(dic)
                    with open("train_label.json", "w") as new_file:
                        json.dump(content,new_file)
                except:
                    with open("train_label.json", "w") as new_file:
                        json.dump(dic,new_file)
                img_id +=1

